## imports

In [129]:
import numpy as np
import pandas as pd
import sklearn 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

## AID FUNCTIONS:

In [152]:
def to_label(num):
    if(num==0): return "T-shirt/top"
    if(num==1): return "Trouser"
    if(num==2): return "Pullover"
    if(num==3): return "Dress"
    if(num==4): return "Coat"
    if(num==5): return "Sandal"
    if(num==6): return "Shirt"
    if(num==7): return "Sneaker"
    if(num==8): return "Bag"
    if(num==9): return "Ankle boot"
    else: return -1
to_label(100)

def to_exp_var(ndarr):
    sum = 0
    for x in ndarr:
        sum +=x
    return sum

## viewing the data and making models :

In [131]:
train_df = pd.read_csv("fashion-mnist_train.csv") ## takes less then 10 sec's
test_df = pd.read_csv("fashion-mnist_test.csv")

### viewing the data: 

In [132]:
train_df.shape ## showing the data's shape
test_df.shape  ##
train_df.columns  ## columns
train_df.head()
train_df.isnull().sum().unique() ## checking if there is null values
test_df.isnull().sum().unique()  ##
train_df.groupby("label").count()["pixel1"] ## or write: "train.label.value_counts()"
test_df.groupby("label").count()["pixel1"]  ## or write: "test.label.value_counts()"

label
0    1000
1    1000
2    1000
3    1000
4    1000
5    1000
6    1000
7    1000
8    1000
9    1000
Name: pixel1, dtype: int64

#### comments:

##### i want to: <br>1. add visualisation for the number of samples in each label. <br>2. add a way to see the photos of the dataset.
##### the missions are in importence-decrecing order. and both of them have solution<br>in one of the notebooks of kaggle in this data set page: https: //www.kaggle.com/code/meetnagadia/cnn-on-fashionmnist#Conclusion

### training the data

In [133]:
## spliting to target and features:
X_train = train_df.iloc[:,1:]
y_train = train_df.iloc[:,0]
X_test = test_df.iloc[:,1:]
y_test = test_df.iloc[:,0]

In [134]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(60000, 784)
(60000,)
(10000, 784)
(10000,)


#### PCA 

In [135]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

## fit the scaler on train only
scaler.fit(X_train)

# apply transform to *train and test*:
X_train_sca = scaler.transform(X_train)
X_test_sca = scaler.transform(X_test)

In [158]:
from sklearn.decomposition import PCA
pca = PCA(196)
pca.fit(X_train_sca)
to_exp_var(pca.explained_variance_ratio_)

0.9297933788412176

In [159]:
X_train_pca = pca.transform(X_train_sca)
X_test_pca = pca.transform(X_test_sca)

#### XGBoost

In [193]:
# from numpy import loadtxt
import xgboost as xgb

# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score

#### KNN

##### classic Knn

###### classic Knn without PCA

In [98]:
    ## all this code takes up to 2:05 min
knn = KNeighborsClassifier().fit(X_train,y_train) ## immediate!!
# print("train score:",knn.score(X_train,y_train)) 
print("test score:",knn.score(X_test,y_test)) 

test score: 0.8691


###### classic Knn with PCA

In [107]:
knn = KNeighborsClassifier().fit(X_train_pca,y_train)
knn.score(X_test_pca,y_test) ## takes up to 15 secs, shorter then before.

0.8283

##### Knn grid_search 

###### Knn grid_search without PCA

In [122]:
        ******takes up to 12 mins!!!******* 
    ## with 5cv 1n and "euclidian" it took 1:10
    ## with 5cv 5n and "minkowski" it took 1:26
    ## whit 6cv 5n and "minkowski" it took 1:16
grid_params = {
    "n_neighbors": [5],
    "weights": ["uniform"],
    "metric" : ["minkowski"]
}
gs = GridSearchCV(KNeighborsClassifier(), grid_params, cv=6) ## immediate!!
gs.fit(X_train,y_train) 

SyntaxError: invalid syntax (Temp/ipykernel_1344/921052143.py, line 1)

In [37]:
# print(gs.best_score_)
# print(gs.best_estimator_)
# print(gs.best_params_)
pd.DataFrame(gs.cv_results_)[['mean_test_score', 'std_test_score', 'params']] ## ridiculous: the diffrence between the 
                                                                              ## scores are unsegnificent

,mean_test_score,std_test_score,params
0,0.850717,0.002896,"{'metric': 'euclidean', 'n_neighbors': 1, 'wei..."
1,0.850717,0.002896,"{'metric': 'euclidean', 'n_neighbors': 1, 'wei..."
2,0.856800,0.002962,"{'metric': 'euclidean', 'n_neighbors': 5, 'wei..."
3,0.857817,0.002975,"{'metric': 'euclidean', 'n_neighbors': 5, 'wei..."
4,0.853967,0.003617,"{'metric': 'euclidean', 'n_neighbors': 10, 'we..."
5,0.855733,0.003143,"{'metric': 'euclidean', 'n_neighbors': 10, 'we..."


In [ ]:
knn_gs = KNeighborsClassifier().fit(X_train,y_train) ## immediate!!
print("train score:",knn.score(X_train,y_train)) 
print("test score:",knn.score(X_test,y_test)) 

###### Knn grid_search with PCA

In [160]:
        # ******takes up to 4:15 min, better! !!!******* 
    ## with 5cv 1n and "euclidian" it took 
    ## with 5cv 5n and "minkowski" it took 
    ## whit 6cv 5n and "minkowski" it took 
grid_params = {
    "n_neighbors": [1,5,10],
    "weights": ["uniform","distance"],
    "metric" : ["minkowski"]
}
gs = GridSearchCV(KNeighborsClassifier(), grid_params, cv=6) ## immediate!!
gs.fit(X_train_pca,y_train) 

GridSearchCV(cv=6, estimator=KNeighborsClassifier(),
             param_grid={'metric': ['minkowski'], 'n_neighbors': [1, 5, 10],
                         'weights': ['uniform', 'distance']})

In [161]:
# print(gs.best_score_)
# print(gs.best_estimator_)
# print(gs.best_params_)
pd.DataFrame(gs.cv_results_)[['mean_test_score', 'std_test_score', 'params']] ## ridiculous: the diffrence between the 
                                                                              ## scores are unsegnificent

,mean_test_score,std_test_score,params
0,0.851083,0.001734,"{'metric': 'minkowski', 'n_neighbors': 1, 'wei..."
1,0.851083,0.001734,"{'metric': 'minkowski', 'n_neighbors': 1, 'wei..."
2,0.862750,0.001528,"{'metric': 'minkowski', 'n_neighbors': 5, 'wei..."
3,0.864267,0.001748,"{'metric': 'minkowski', 'n_neighbors': 5, 'wei..."
4,0.863183,0.001704,"{'metric': 'minkowski', 'n_neighbors': 10, 'we..."
5,0.863833,0.001937,"{'metric': 'minkowski', 'n_neighbors': 10, 'we..."


In [187]:
knn_gs = KNeighborsClassifier().fit(X_train_pca,y_train) ## immediate!!
# print("train score:",knn.score(X_train_pca,y_train)) 
# print("test score:",knn.score(X_test,y_test)) 
knn_gs.score(X_test_pca,y_test)


0.8691

#### logistic regretion

##### classic log reg

In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train,y_train) ## takes up to 7 secs

C:\Users\noams\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
logreg.score(X_test,y_test)

0.8544

##### grid search log reg

###### comment: <br> check how to do grid search on logistic regresstion on the fmnist <br> check how to "cv 2" but with 0.10 test group each time

In [ ]:
logreg = LogisticRegression()

param_grid = {
        "solver": ["saga"]
        # "penalty": ["none", "l1","l2", "elasticnet"],
        # "C" : [100, 10, 1.0, 0.1, 0.01]

        #              >1:30       1:40      >2:30       1:15  >4:30
        # "solver": ["newton-cg", "lbfgs", "liblinear", "sag","saga"],
        # "penalty": ["none", "l1","l2", "elasticnet"],
        # "C" : [100, 10, 1.0, 0.1, 0.01]
}
print(param_grid)

gs = GridSearchCV(LogisticRegression(),param_grid=param_grid,scoring=accuracy_score,cv=10)
gs.fit(X,y)

In [ ]:
print("hi")

hi


In [ ]:
logreg.score(X_test,y_test)

0.8487777777777777

### i tried to chek if the comment "indicating the lightness or darkness of that pixel, _with higher numbers meaning darker_" is wrong. i think it is... the following cell is the try to debunk:

In [ ]:
# print(train_df.pixel1.unique())
# print(train_df.pixel2.unique())
# print(train_df.pixel3.unique())
# train_df["label" == 0] ## checking the snickers, do know why but it makes an exception.